In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

MODEL_NAME = "bert-base-uncased"  
BATCH_SIZE = 16
MAX_LEN = 64
TEST_SIZE = 0.2
RANDOM_STATE = 42

csv_path = "../../data/metadata.csv"
print(f"📂 Đang load dataset: {csv_path}")
df = pd.read_csv(csv_path)


texts = df["notes"].astype(str).tolist()
labels = df["finding"].astype("category").cat.codes
label_names = list(df["finding"].astype("category").cat.categories)

print(f"✅ Số mẫu: {len(texts)} | Số nhãn: {len(label_names)} | Nhãn: {label_names}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print(f"🧠 Dùng thiết bị: {device}")

def get_bert_embeddings(texts, batch_size=BATCH_SIZE, max_len=MAX_LEN):
    """Tạo CLS embedding cho danh sách văn bản."""
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="🔹Đang tạo embedding"):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=max_len
        ).to(device)

        with torch.no_grad():
            outputs = model(**enc)
            cls_embeds = outputs.last_hidden_state[:, 0, :]  # vector CLS
            embeddings.append(cls_embeds.cpu().numpy())

    return np.vstack(embeddings)

print("🚀 Đang tính toán BERT embeddings...")
X = get_bert_embeddings(texts)
y = labels.values
print(f"✅ Embeddings shape: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

results = {}
trained_models = {}

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
results["Logistic Regression"] = {
    "accuracy": accuracy_score(y_test, y_pred_lr),
    "f1": f1_score(y_test, y_pred_lr, average="weighted"),
    "report": classification_report(y_test, y_pred_lr, target_names=label_names, digits=4)
}
trained_models["Logistic Regression"] = lr

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
results["Random Forest"] = {
    "accuracy": accuracy_score(y_test, y_pred_rf),
    "f1": f1_score(y_test, y_pred_rf, average="weighted"),
    "report": classification_report(y_test, y_pred_rf, target_names=label_names, digits=4)
}
trained_models["Random Forest"] = rf

print("\n==== 🧾 KẾT QUẢ (BERT + ML) ====\n")
for name, res in results.items():
    print(f"--- {name} ---")
    print("Accuracy:", res["accuracy"])
    print("F1-weighted:", res["f1"])
    print(res["report"])

save_dir = "../models"
os.makedirs(save_dir, exist_ok=True)

for name, model_obj in trained_models.items():
    file_path = os.path.join(save_dir, f"bert_{name.replace(' ', '_').lower()}.pkl")
    with open(file_path, "wb") as f:
        pickle.dump(model_obj, f)
    print(f"✅ Saved model: {file_path}")

np.save(os.path.join(save_dir, "bert_embeddings.npy"), X)
print("✅ Saved embeddings.")

save_results_path = os.path.join(save_dir, "results_bert.json")
with open(save_results_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)
print(f"✅ Saved evaluation results to {save_results_path}")

for name, model_obj in trained_models.items():
    print(f"\n🔍 Model: {name}")
    print("Classes learned:", model_obj.classes_)
    print("Number of classes:", len(model_obj.classes_))


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Đang load dataset: ../../data/metadata.csv
✅ Số mẫu: 3400 | Số nhãn: 3 | Nhãn: ['COVID-19', 'NORMAL', 'PNEUMONIA']
🧠 Dùng thiết bị: cpu
🚀 Đang tính toán BERT embeddings...


🔹Đang tạo embedding: 100%|██████████| 213/213 [00:39<00:00,  5.44it/s]


✅ Embeddings shape: (3400, 768)

==== 🧾 KẾT QUẢ (BERT + ML) ====

--- Logistic Regression ---
Accuracy: 0.8720588235294118
F1-weighted: 0.8718168401273034
              precision    recall  f1-score   support

    COVID-19     0.8419    0.9211    0.8797       266
      NORMAL     0.8920    0.8351    0.8626       188
   PNEUMONIA     0.8967    0.8451    0.8702       226

    accuracy                         0.8721       680
   macro avg     0.8769    0.8671    0.8708       680
weighted avg     0.8740    0.8721    0.8718       680

--- Random Forest ---
Accuracy: 0.875
F1-weighted: 0.8749274026204674
              precision    recall  f1-score   support

    COVID-19     0.8815    0.8947    0.8881       266
      NORMAL     0.8791    0.8511    0.8649       188
   PNEUMONIA     0.8640    0.8717    0.8678       226

    accuracy                         0.8750       680
   macro avg     0.8749    0.8725    0.8736       680
weighted avg     0.8750    0.8750    0.8749       680

✅ Saved model